# Muestreo de distribuciones generales

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/b/bf/Simple_random_sampling.PNG" width="400px" height="400px" />

> Ya nos hicimos una idea básica de lo que es la estimación Montecarlo. Vimos que es fundamental poder muestrear las distribuciones, y por ello vimos métodos para muestrear distribuciones en una dimensión, tanto discretas como continuas.

> Ahora, ¿Qué pasa si nuestra distribución es multivariable? Bien, en este notebook estaremos viendo métodos generales para muestrear cualquier distribución através de cadenas de Markov.

> **Objetivos:**
> - Estudiar las cadenas de Markov, sus componentes y propiedades básicas.
> - Entender las cadenas de Markov como artefactos para muestrar distribuciones.
> - Comprender la técnica de muestreo de Gibbs.
> - Comprender la técnica de muestreo de Metropolis-Hastings.
> - Comprender la técnica de muestreo de Montecarlo Hamiltoniano.

> **Referencias:**
> - Bayesian Methods for Machine Learning course, HSE University, Coursera.
> - Probabilistic Graphical Models: Principles and Techniques, By Daphne Koller and Nir Friedman. Ch. 12.

<p style="text-align:right;"> Imagen recuperada de: https://upload.wikimedia.org/wikipedia/commons/b/bf/Simple_random_sampling.PNG.</p>

___

## 1. Cadenas de Markov

Una cadena de Markov es un **modelo de transiciones probabilístico** sobre ciertos estados $x$. Para ello definimos la probabilidad de transicionar de un estado $x$ a otro estado $x'$ como $T(x \to x')$, y al ser una probabilidad debe cumplir que:

$$
\forall x: \sum_{x'}T(x \to x') = 1.
$$

#### Ejemplo.

Consideramos la trayectoria de un borracho modelada por la siguiente cadena de Markov:

![](figures/drunkMC.png)

Cada arco representa la probabilidad de transicionar de un estado a otro (incluyendo al mismo estado de partida). Por ejemplo, en la cadena de Markov de arriba:

$$
T(0 \to 0) = 0.5, \quad T(0 \to 1) = 0.25, \quad T(0 \to -1) = 0.25, \quad T(0 \to -2) = T(0 \to 2) = 0
$$

Es decir, si el borracho está en la posición $0$, tiene un 50% de probabilidad de permanecer allí, y un 25% de moverse a las posiciones $1$ y $-1$.

Una pregunta natural es:

- En determinado instante $t$, ¿Cuál es la probabilidad $p^{(t)}(x^{(t)})$ de que el borracho esté en determinada posición?

La respuesta es:

- Dada una distribución inicial de probabilidad $p^{(0)}(x^{(0)})$, podemos encontrar $p^{(t)}(x^{(t)})$ como:

  $$
  p^{(t+1)}(x^{(t+1)} = x') = \sum_{x} p^{(t)}(x^{(t)}) T(x \to x').
  $$

Por ejemplo, si sabemos que el borracho comenzó en la posición $0$, tenemos que:

| $p^{(t)}(x^{(t)})$ | $x^{(t)}=-2$ | $x^{(t)}=-1$ | $x^{(t)}=0$ | $x^{(t)}=1$ | $x^{(t)}=2$ |
| ------------------ | ------------ | ------------ | ----------- | ----------- | ----------- |
| $p^{(0)}(x^{(0)})$ | $0$          | $0$          | $1$         | $0$         | $0$         |
| $p^{(1)}(x^{(1)})$ | $0$          | $0.25$       | $0.5$       | $0.25$      | $0$         |
| $p^{(2)}(x^{(2)})$ | $0.0625$     | $0.25$       | $0.375$     | $0.25$      | $0.0625$    |

**Explicar estos valores en el pizarrón.**

Esta tabla la podemos continuar todo lo que queramos, y veremos que después de algunas iteraciones **todos los valores convergerán a $0.2$**.

In [1]:
# Importamos numpy


In [2]:
# Matriz de transición


# Distribución inicial


In [ ]:
# Distribución 1


In [ ]:
# Distribución 2


In [ ]:
# Equivalentemente (productos sucesivos)


In [ ]:
# Equivalentemente (potenciación de la matriz de transición)


In [ ]:
# Distribución 100


In [ ]:
# Distribución 1000


**¿Cómo podemos interpretar esto?**

Supongamos que dejamos caminar al borracho durante varios pasos (digamos 1000) y obtenemos la siguiente secuencia:

$$
0, 0, 1, 2, 1, \dots, -1
$$

Podemos hacer lo mismo repetidas veces:

$$
0, 1, 1, 0, 1, \dots, 2
$$

$$
0, 1, 0, -1, 0, \dots, 0
$$

$$
0, 1, 0, -1, -2, \dots, 1
$$

$$
0, -1, -2, -2, -1, \dots, -2
$$

Entonces, podemos tomar las últimas observaciones en cada caso ($-1$, $2$, $0$, $1$, y $-2$) que corresponden a muestras de la distribución del paso número 1000 dado que el borracho comenzó en la posición 0.

Como vimos anteriormente, estas muestras corresponderán a la distribución discreta uniforme:

$$
p(x = i) = 0.2, \quad \forall i \in \{-2, -1, 0, 1, 2\}.
$$

Esta es una manera muy extravagante de generar muestras de esta distribución, pues ya vimos en el notebook pasado que hay maneras mucho más sencillas de hacerlo.

Sin embargo, esto nos da una idea de qué hacer en casos más complejos: **podemos construir una cadena de markov que converja a una distribución deseada y después tomar las muestras de allí**.

Es decir, queremos muestrear de una distribución $p(x)$:

1. Construimos una cadena de Markov que converge a $p(x)$.

2. Comenzamos desde algún punto inicial $x_0$.

3. Simulamos la cadena de Markov por muchas muestras:
   
   $$
   x_k \sim T(x_{k-1} \to x_k)
   $$
   
4. Si dejamos pasar suficientes muestras, las siguientes $x_k$ corresponderán a muestras de la distribución deseada $p(x)$.

Una pregunta natural es:

**¿Siempre converge una cadena de Markov?**

La respuesta es no siempre.

Para empezar a hablar de convergencia, primero definimos:

> *Definición.* Una distribución $\pi$ es estacionaria en la cadena de Markov con transición $T$ si:
>
> $$\pi(x') = \sum_{x} \pi(x) T(x \to x').$$

Básicamente si de un instante de tiempo al siguiente la distribución no cambia, entocnes es estacionaria.

> *Definition (Regular Markov chain).* Una cadena de Markov se llama **regular** si existe $k\in\mathbb{N}$ tal que, para cada par $x, x'$, la probabilidad de llegar de $x$ a $x'$ en $k$ pasos es positiva ($>0$).

> *Teorema.* Una cadena de Markov regular converge siempre a una única distribución $\pi(x)$ sin importar la distribución inicial.

Por ejemplo, la cadena de Markov del borracho es regular:

In [ ]:
# Distribución inicial diferente


Entonces ya sabemos definir cadenas de Markov convergentes. Ahora, **¿Cómo hacer para que converjan a una distribución deseada?**

## 2. Muestreo de Gibbs

Para responder a la anterior pregunta, surge la metodología del muestreo de Gibbs.

Supongamos que tenemos una distribución (no normalizada) de tres dimensiones (puede tener dimensiones arbitrarias):

$$
p(x_1, x_2, x_3) = \frac{\tilde{p}(x_1, x_2, x_3)}{Z}
$$

**Idea:**

1. Comenzamos con un punto inicial $(x_1^0, x_2^0, x_3^0)$.

2. Muestreamos de la siguiente manera:

   $$
   x_1^1 \sim p(x_1 | x_2^0, x_3^0)
   $$
   
   $$
   x_2^1 \sim p(x_2 | x_1^1, x_3^0)
   $$
   
   $$
   x_3^1 \sim p(x_3 | x_1^1, x_2^1)
   $$
   
   - Notemos que vamos reemplazando las muestras que vamos generando inmediatamente. No se puede paralelizar.
   - Muestreo de distribuciones unidimensionales.
   - Si conocemos solo la distribución no normalizada, no hay problema: $p(x_1 | x_2^0, x_3^0) = \frac{\tilde{p}(x_1, x_2^0, x_3^0)}{Z_1}$

3. Y así sucesivamente:

   $$
   x_1^{k} \sim p(x_1 | x_2^{k-1}, x_3^{k-1})
   $$
   
   $$
   x_2^{k} \sim p(x_2 | x_1^{k}, x_3^{k-1})
   $$
   
   $$
   x_3^{k} \sim p(x_3 | x_1^{k}, x_2^{k})
   $$

### ¿Por qué funciona?

Veámoslo desde el punto de vista de redes de Markov. Queremos probar que $p$ es una distribución estacionaria:

$$
p(x', y', z') = \sum_{x, y, z} T(x, y, z \to x', y', z') p(x, y, z).
$$

Para el muestreo de Gibbs, tenemos que la transición es:

$$
T(x, y, z \to x', y', z') = p(x' | y, z) p(y' | x', z) p(z' | x', y').
$$

Concentrémonos en el lado derecho de la igualdad:

\begin{align}
\sum_{x, y, z} p(x' | y, z) p(y' | x', z) p(x, y, z) & =  p(z' | x', y')\sum_{y, z} p(x' | y, z) p(y' | x', z) \underbrace{\sum_{x} p(x, y, z)}_{p(y, z)} \\
& = p(z' | x', y')\sum_{z} p(y' | x', z) \underbrace{\sum_{y} \underbrace{p(x' | y, z) p(y, z)}_{p(x', y, z)}}_{p(x', z)} \\
& = p(z' | x', y')\underbrace{\sum_{z} \underbrace{p(y' | x', z) p(x', z)}_{p(x', y', z)}}_{p(x', y')} \\
& = p(z' | x', y')p(x', y') \\
& = p(x', y', z').
\end{align}

Tal y como queríamos.

**Comentarios:**

- Esta misma idea se puede replicar para un número arbitrario de dimensiones.

- También funciona en el caso continuo, reemplazando sumas por integrales.

### Ejemplo

Tomado del [siguiente enlace](https://towardsdatascience.com/gibbs-sampling-8e4844560ae5)

Supongamos que queremos muestrear la siguiente distribución:

$$
p(x, y) = \frac{1}{Z} \exp \left\{- \frac{x^2 y^2 + x^2 + y^2 - 8x - 8y}{2} \right\}
$$

In [12]:
# Importamos matplotlib.pyplot
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
# Importamos numpy
import numpy as np

In [13]:
def rare_pdf(x, y):
    return np.exp(-(x**2 * y**2 + x**2 +y**2 - 8 * x - 8 * y) / 2)

In [ ]:
# Gráfico
x = np.linspace(-1, 6, 100)
y = np.linspace(-1, 6, 100)
x, y = np.meshgrid(x, y)
z = rare_pdf(x, y)
fig = plt.figure(figsize=(8, 4))
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(x, y, z)

ax2 = fig.add_subplot(122)
ax2.contour(x, y, z)
plt.show()

Recordemos que necesitamos las distribuciones condicionales:

$$
p(x | y) = \frac{p(x, y)}{p(y)} = \frac{1}{Z(y)} \exp\left\{-\left(x - \frac{4}{1 + y^2}\right)^2 \frac{1}{2\frac{1}{1+y^2}}\right\} \propto \mathcal{N}\left(x\left\lvert \frac{4}{1 + y^2}, \frac{1}{1 + y^2}\right.\right)
$$

$$
p(y | x) = \frac{p(x, y)}{p(x)} = \frac{1}{Z(x)} \exp\left\{-\left(y - \frac{4}{1 + x^2}\right)^2 \frac{1}{2\frac{1}{1+x^2}}\right\} \propto \mathcal{N}\left(y\left\lvert \frac{4}{1 + x^2}, \frac{1}{1 + x^2}\right.\right)
$$

In [15]:
# Punto inicial
x0 = np.array([0, 0])

In [20]:
def gibbs_sampling(x0, M):
    
    # Prealloc de variables
    xs = np.zeros((M + 1, len(x0)))
    xs[0, :] = x0
    
    for k in range(M):
        xk, yk = xs[k, :]
        
        # Muestreamos x^{k+1} de la condicional p(x | y^{k})
        
        
        # Muestreamos y^{k+1} de la condicional p(y | x^{k+1})
        
        
        # Guardamos siguiente muestra
        xs[k + 1, 0] = xk1
        xs[k + 1, 1] = yk1
        
    return xs

In [21]:
xs = gibbs_sampling(x0, 1000)

In [ ]:
xs.shape

In [ ]:
# Gráfico
plt.figure(figsize=(5, 5))
x = np.linspace(-1, 6, 100)
y = np.linspace(-1, 6, 100)
x, y = np.meshgrid(x, y)
z = rare_pdf(x, y)
plt.contour(x, y, z)
plt.scatter(xs[:, 0], xs[:, 1], alpha=0.3)
plt.show()

Como idea importante, podemos observar que nunca conocimos la distribución $p(x,y)$ normalizada.

## 3. Metropolis-Hastings

El muestreo de Gibbs parece hacer un buen trabajo. Sin embargo, por la manera en que se obtienen las muestras, éstas están muy correlacionadas, y recordemos que muchos análisis los basamos en el supuesto de iid.

Así que en esta sección estudiaremos un método para producir muestras no correlacionadas. La idea básica de este método es generalizar la técnica de muestreo y rechazo para distribuciones en general.

**Idea**

1. Comenzamos con un punto inicial $(x_1^0, x_2^0, x_3^0)$.

2. Muestreamos $x^{k+1}$ de una distribución $Q(x^{k} \to x')$, que en principio no tiene nada que ver con la distribución deseada $p(x)$, sino que está diseñada para explorar el espacio (evitar correlación).

3. Aceptamos la muestra $x'$ con probabilidad $A(x^{k} \to x')$.
   - Dado que $Q(x^{k} \to x')$ quiere explorar tooodo el espacio, debemos evitar que muchas muestras caigan en una región de baja probabilidad de la distribución deseada $p(x)$.
   
   De manera que la transición es:
   $$
   T(x \to x') = Q(x \to x') A(x \to x').
   $$
   
   - Si no se acepta la muestra, hacemos $x^{k+1}=x^k$

**¿Cómo elegimos la probabilidad de aceptación $A(x \to x')$?**

Bueno, pues debemos de asegurarnos que la distribución deseada $p(x)$ sea estacionaria:

$$
p(x') = \sum_{x} p(x) T(x \to x').
$$

Sin embargo esta condición es compleja de satisfacer directamente, e involucraría restricciones sobre $A$ y $Q$.

La idea es que se elija $Q$ independientemente, y después de eso, diseñar $A$ para que todo funcione correctamente.

Hay una condición que implica estacionariedad, y es la siguiente:

> *Teorema*. Si $p(x) T(x \to x') = p(x') T(x' \to x)$, entonces
  $$
p(x') = \sum_{x} p(x) T(x \to x').
$$

> *Prueba.*
> Observamos que
  $$
  \sum_{x} p(x) T(x \to x') = \sum_{x} p(x') T(x' \to x) = p(x') \underbrace{\sum_x T(x' \to x)}_{1}.
  $$

Por tanto, si elegimos $A$ de manera que se satisfaga $p(x) T(x \to x') = p(x') T(x' \to x)$, aseguraremos que $p(x)$ sea la distribución estacionaria deseada.

Veamos:

\begin{align}
            & p(x) T(x \to x') = p(x') T(x' \to x) \\
\Rightarrow & p(x) Q(x \to x') A(x \to x') = p(x') Q(x' \to x) A(x' \to x) \\
\Rightarrow & \frac{A(x \to x')}{A(x' \to x)} = \frac{p(x') Q(x' \to x)}{p(x) Q(x \to x')} = \rho
\end{align}

Esta razón la podemos calcular dado que $p(x)$ es la distribución deseada, y $Q$ la proponemos nosotros.

Ahora, tenemos dos posibildades:

1. $\rho \leq 1$ 

   En este caso, podemos asignar:
   
   $$
   A(x \to x') = \rho, \quad A(x' \to x)=1,
   $$
   
   de modo que $\frac{A(x \to x')}{A(x' \to x)} = \rho$.
   
2. $\rho > 1$ 

   En este caso, podemos asignar:
   
   $$
   A(x \to x') = 1, \quad A(x' \to x)=\frac{1}{\rho},
   $$
   
   de modo que $\frac{A(x \to x')}{A(x' \to x)} = \rho$.
   
   
De esta manera, la selección de $A$ será en general:

$$
A(x \to x') = \min \left\{1, \frac{p(x') Q(x' \to x)}{p(x) Q(x \to x')}\right\}
$$

De nuevo, observamos que con esta selección, no necesitamos conocer $p(x)$ normalizada, dado en la razón se cancela la constante de normalización.

**Importante**

1. $Q$ debe ser una distribución **positiva**.

2. Debemos elegir $Q$ para explorar el espacio de estados exhaustivamente, para generar muestras independientes. Sin embargo, si $Q$ explora mucho las regiones de baja densidad de $p$ entonces tendremos poca probabilidad de aceptación.

### Ejemplo

De nuevo, queremos muestrear la distribución:

$$
p(x_1, x_2) = \frac{1}{Z} \exp \left\{- \frac{x_1^2 x_2^2 + x_1^2 + x_2^2 - 8x_1 - 8x_2}{2} \right\}
$$

Proponemos la siguiente distribución $Q(x \to x')$:

$$
Q(x \to x') = \mathcal{N}\left(x'\left\lvert x, I\right.\right) = \mathcal{N}\left(x\left\lvert x', I\right.\right) = Q(x' \to x),
$$

la cual es simétrica.

De esta manera tenemos que:

$$
A(x \to x') = \min \left\{1, \frac{p(x') Q(x' \to x)}{p(x) Q(x \to x')}\right\} = \min \left\{1, \frac{p(x')}{p(x)}\right\}
$$

In [25]:
# Punto inicial
x0 = np.array([0, 0])

In [ ]:
from scipy.stats import multivariate_normal

In [27]:
def metropolis_hastings(x0, M):
    
    # Prealloc de variables
    xs = np.zeros((M + 1, len(x0)))
    xs[0, :] = x0
    
    for k in range(M):
        xk = xs[k, :]
        
        # Muestreamos x' de Q(x -> x')
        
        
        # Evaluamos A
        
        
        # Guardamos siguiente muestra
        if np.random.uniform() < A:
            xs[k + 1, :] = xk1
        else:
            xs[k + 1, :] = xk
        
    return xs

In [33]:
xs = metropolis_hastings(x0, 10000)

In [ ]:
xs

¿Cuántos no repetidos?

In [35]:
xsu = np.unique(xs, axis=0)

In [ ]:
xsu.shape

In [ ]:
# Gráfico
plt.figure(figsize=(5, 5))
x = np.linspace(-1, 6, 100)
y = np.linspace(-1, 6, 100)
x, y = np.meshgrid(x, y)
z = rare_pdf(x, y)
plt.contour(x, y, z)
plt.scatter(xsu[:, 0], xsu[:, 1], alpha=0.3)
plt.show()

¿Cómo cambian los $x_s$ con la variabilidad de $Q$?

In [ ]:
plt.plot(xs[-200:])

**Comentarios**

1. Similarmente, no necesitamos conocer la distribución normalizada.

2. Es una generalización del algoritmo de muestreo y rechazo para distribuciones en general.

3. Mucho más fácil de implementar que el muestreo de Gibbs, dado que no tenemos que calcular las distribuciones condicionales.

4. Pero debemos elegir una distribución $Q$ apropiada.

## 4. MCMC Hamiltoniano



> Parece ser un principio general que, siempre que hay una forma aleatoria de hacer algo, existe una forma no aleatoria que da mejores resultados, pero requiere más pensamiento. - [E. T. Jaynes](https://es.wikipedia.org/wiki/Edwin_Thompson_Jaynes)

El algoritmo de Metropolis-Hastings parece bastante bueno. Es fácil de implementar, una vez se diseña una distribución de exploración $Q$. Acá el [artículo original](https://bayes.wustl.edu/Manual/EquationOfState.pdf)

Sin embargo, al ser un algoritmo de muestreo y rechazo, muchas veces se descartan demasiadas muestras, lo que lo lleva a ser ineficiente. En el ejemplo de arriba, nos quedamos con solamente ~350 muestras únicas de las 1000 generadas.

Haciendo referencia a la acotación de Jaynes, parece que el algoritmo de Gibbs es menos aleatorio, dado que se explota el conocimiento de la distribución al costo de tener que calcular las distribuciones condicionales. Sin embargo, el algoritmo de Gibbs tiene el problema que las muestras consecutivas poseen una correlación alta.

Dado lo anterior, han habido varias contribuciones e innovaciones en la última década. Un camino de innovación ha sido la interpretación física de las distribuciones de probabilidad, incluyendo el uso de gradientes. Esto se conoce como **Montecarlo Hamiltoniano** (HMC). 

Este método, lleva la acotación de Jaynes más allá. Su costo computacional es mucho más alto comparado con los algoritmos de Gibbs o Metrópolis, pero las muestras que se obtienen son mucho más eficientes en el sentido de que no están tan correlacionadas, y que no se rechaza ninguna de ellas. A final de cuentas el tiempo de cómputo necesario comúnmente resulta ser menor con HMC, porque aunque cada muestra de HMC sea más costosa, no se descarta niguna. Esto es especialmente relevante cuando tenemos modelos complejos con muchos parámetros.

### ¿Cómo funciona HMC?

El HMC hace un simil de la distribución de probabilidad como si fuera un fenómeno físico. Tenemos "una partícula que se va a mover sobre la distribución de probabilidad", y a intervalos fijos "de tiempo" se toman las muestras.

En cada punto de muestra, se asigna un momentum o velocidad inicial, y se "deja" que la partícula se mueva sobre la distribución hasta que llegue el momento de tomar otra muestra.

Para generar esta "simulación" necesitamos:

1. La distribución de probabilidad negativa logarítmica $U$.
2. El gradiente de la distribución de probabilidad negativa logarítmica $\nabla U$. Este se utilizará para determinar el movimiento de la partícula.
3. El tamaño de paso $\epsilon$. La simulación no se hace de forma continua, sino discreta.
4. El número fijo de pasos $L$ al que se toman las muestras.
5. La muestra inicial.

Siguiendo con el ejemplo anterior:

$$
p(x, y) = \frac{1}{Z} \exp \left\{- \frac{x^2 y^2 + x^2 + y^2 - 8x - 8y}{2} \right\},
$$

tenemos que $U(x, y) = - \log p(x, y)$:

$$
U(x, y) = \log(Z) + \frac{x^2 y^2 + x^2 + y^2 - 8x - 8y}{2}.
$$

con esto:

$$
\frac{\partial U}{\partial x} = x (y^2 + 1)  - 4 \qquad \frac{\partial U}{\partial y} = y (x^2 + 1) - 4
$$


In [42]:
import numpy as np

In [43]:
def U(x):
    x, y = x[0], x[1]
    return (x**2 * y**2 + x**2 + y**2 - 8 * x - 8 * y) / 2

def grad_U(x):
    x, y = x[0], x[1]
    return np.array([
        x * (y**2 + 1) - 4,  # dU / dx
        y * (x**2 + 1) - 4   # dU / dy
    ])

In [45]:
def hmc(U, grad_U, epsilon, L, current_q):
    q = current_q
    
    # Momentum
    p = np.random.normal(size=len(q))
    current_p = p
    
    # Alinear el momentum con el modelo
    p = p - epsilon * grad_U(q) / 2
    
    # Inicialización de la trayectoria
    qtraj = np.zeros((L + 1, len(q)))
    ptraj = np.zeros((L + 1, len(q)))
    qtraj[0, :] = current_q
    ptraj[0, :] = p
    
    # Actualizar posición y velocidad
    for i in range(1, L + 1):
        q = q + epsilon * p  #  Cambio en la posición
        # Make a full step for the momentum, except at end of trajectory
        if i < L:
            p = p - epsilon * grad_U(q)
            ptraj[i] = p
        qtraj[i] = q

    # Alinear velocidad final con el modelo
    p = p - epsilon * grad_U(q) / 2
    ptraj[L, :] = p

    # Evaluar energías cinética y potencial al inicio y final de la trauectoria
    current_U = U(current_q)
    current_K = (current_p**2).sum() / 2
    proposed_U = U(q)
    proposed_K = (p**2).sum() / 2
    # Acepta o rechaza el estado al final de la trayectoria:
    # Excepcionalmente se admite un aumento pequeño de energía
    accept = 0
    new_q = current_q
    if np.random.uniform() < np.exp(current_U - proposed_U + current_K - proposed_K):
        new_q = q  # Accept
        accept = 1
    return new_q, qtraj, ptraj, accept

In [ ]:
# Valores
q = np.ones(2)
L = 23
n_samples = 1000
Q = np.zeros((n_samples + 1, len(q)))
Q[0, :] = q
epsilon = 0.05
# Simulación
for i in range(1, n_samples + 1):
    Q[i, :], _, _, _ = hmc(U, grad_U, epsilon, L, Q[i - 1, :])

Qu = np.unique(Q, axis=0)
Qu.shape

In [ ]:
# Gráfico
plt.figure(figsize=(5, 5))
x = np.linspace(-1, 6, 100)
y = np.linspace(-1, 6, 100)
x, y = np.meshgrid(x, y)
z = rare_pdf(x, y)
plt.contour(x, y, z)
plt.scatter(Qu[:, 0], Qu[:, 1], alpha=0.3)
plt.show()

De lo anterior, podemos ver que el HMC es por mucho el más complejo de programar, pero trae como ventaja que es muy eficiente en cuanto al número de muestras; además estas muestras son poco correlacionadas.

La idea básica es que las muestras son partículas que partiendo de un punto inicial con una velocidad inicial, se deslizan a través de la distribución de probabilidad (siguiendo las ecuaciones de movimiento) para generar la siguiente muestra.

Sintonizar los parámetros $\epsilon$ y $L$ puede resultar una tarea interesante, y de ellos depende la calidad de las muestras que obtenemos.

En la práctica `pymc` hará todo el trabajo por nosotros. Ya están los métodos de HMC programados con la selección automática de los parámetros, que nos generarán, en la mayoría de los casos, muestras bastante competentes. El tipo de algoritmo que usa pymc por defecto es un NO-U-TURN SAMPLER o NUTS.

Sin embargo, conviene entender un poco los conceptos detrás de estos algoritmos.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>